In [45]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.common.exceptions import StaleElementReferenceException
import re
import time
import csv

In [46]:
driver = webdriver.Chrome()
url = "https://underwood1.yonsei.ac.kr/com/lgin/SsoCtr/initExtPageWork.do?link=handbList&locale=eng"
driver.get(url)


In [47]:
# 대학(원) 분류 및 개설 학과 등등 조건 설정
labels = driver.find_element(By.CLASS_NAME, "cl-layout-content")

# 대학(원) 분류 선택 태그 id: uuid-5i
classification_combobox = labels.find_element(By.ID, "uuid-5i")
cc_combobox_button = classification_combobox.find_element(By.CLASS_NAME, "cl-combobox-button")
opened_course_combobox = labels.find_element(By.ID, "uuid-5k")
opened_course_button = opened_course_combobox.find_element(By.CLASS_NAME, "cl-combobox-button")

# driver.execute_script(f'arguments[0].setAttribute("{attribute_name}", "{attribute_value}")', element)


# 개설학과 분류 선택 태그 id: uuid-5k


In [48]:

def get_courses():
    parent_container = driver.find_element(By.CLASS_NAME, "cl-grid-centersplit")
    child_elements = parent_container.find_elements(By.XPATH,"./child::div")[1]
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    child_elements = child_elements.find_element(By.XPATH,"./child::div")
    scroll_origin = ScrollOrigin.from_element(child_elements)
    courses = child_elements.find_elements(By.XPATH,'.//div[@role="row"]')
    return [courses, scroll_origin]
    

In [49]:
xpath_expression = f"//*[contains(text(), '{"조회"}')]"
search_button = labels.find_elements(By.XPATH, xpath_expression)[0]


# 8개씩 나오는데 내려가면서 다 봐야 함.     

In [50]:
# 총건수
def get_search_nums():
    search_div = driver.find_element(By.XPATH, '//div[@aria-label[contains(., "총건수")]]')
    search_nums = search_div.get_attribute("aria-label")
    search_nums =  re.search(r'\[([\d]+)\]', search_nums)
    search_nums = int(search_nums.group(1))
    return search_nums

In [51]:
# 추출 템플릿 및 dictionary 함수화 시켜야 함. 
def extract_course(course):
    columns = course.find_elements(By.XPATH, "./*")
    semester = columns[1].text
    campus = columns[2].text
    major = columns[3].text
    grade = columns[4].text
    subject_type = columns[5].text
    session = columns[6].text
    course_code = columns[7].text
    credits = columns[12].text
    course_name = columns[13].text
    is_cancelled = columns[14].text
    instructor = columns[15].text
    schedule = columns[16].text
    classroom = columns[17].text

    

    course_info = {
        'semester': semester,
        'campus': campus,
        'major': major,
        'grade': grade,
        'subject_type': subject_type,
        'session': session,
        'course_code': course_code,
        'credits': credits,
        'course_name': course_name,
        'is_cancelled': is_cancelled,
        'instructor': instructor,
        'schedule': schedule,
        'classroom': classroom
    }


    return course_info


In [52]:
fieldnames = ['semester',
                'campus',
                'major',
                'grade',
                'subject_type',
                'session',
                'course_code',
                'credits',
                'course_name',
                'is_cancelled',
                'instructor',
                'schedule',
                'classroom']

In [53]:

prev_oc_id = None
oc_id = None
for i in range(35):
    cc_combobox_button.send_keys(Keys.DOWN)
    
    # keys_down (클릭한 상태에서 눌렀을 때 )
    opened_course_button.click()
    while (True):
        opened_course_button.send_keys(Keys.DOWN)
        opened_course_button.click()
        oc_id = opened_course_combobox.find_element(By.TAG_NAME, "input").get_attribute("aria-activedescendant")
        if (prev_oc_id is not None and prev_oc_id == oc_id):
            prev_oc_id = None
            oc_id = None
            break
        # 검색하고 결과값 크롤링 => csv 파일에 넣기!
        

        # 검색
        search_button.click()
        time.sleep(3)
        courses = get_courses()
        search_nums = get_search_nums()
        done = True
        current_idx = 0
        with open("output.csv", "a+", newline='') as csv_file:
            
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            while done:
                try:
                    courses, scroll_origin = get_courses()
                    if (current_idx == search_nums):
                        done = False
                    for course in courses:
                        course_idx = int(course.get_attribute("aria-rowindex"))
                        if (course_idx > current_idx):
                            writer.writerow(extract_course(course))
                            current_idx += 1
                    ActionChains(driver)\
                    .scroll_from_origin(scroll_origin, 0, 480)\
                    .perform()
                    time.sleep(1)  # 스크롤이 완료될 때까지 잠시 대기
            
            
                except StaleElementReferenceException:
                    pass

        print("Section Completed ✅")


        prev_oc_id = oc_id
            


Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅
Section Completed ✅


KeyboardInterrupt: 